In [ ]:
pip install transformers

In [ ]:
pip install torch

# BETO

In [1]:
from transformers import pipeline

# 1. Fill-Mask (Completado de texto con máscara)

In [3]:

fill_mask_pipe = pipeline("fill-mask", model="dccuchile/bert-base-spanish-wwm-uncased")
frase_fill_mask = "El cambio climático es [MASK] para el futuro."
resultados_fill_mask = fill_mask_pipe(frase_fill_mask)

print("Resultados para 'fill-mask':")
for res in resultados_fill_mask:
    print(f"Palabra predicha: {res['token_str']} | Score: {res['score']:.4f}")
print()

Resultados para 'fill-mask':
Palabra predicha: necesario | Score: 0.1978
Palabra predicha: bueno | Score: 0.1463
Palabra predicha: esencial | Score: 0.1131
Palabra predicha: importante | Score: 0.0878
Palabra predicha: vital | Score: 0.0630



# 2. Text Classification (Clasificación de texto)

Se utiliza para clasificar el texto. En este ejemplo, se clasifica el sentimiento de un texto como positivo, negativo o neutral.
1 estrella: Muy negativo.
2 estrellas: Negativo.
3 estrellas: Neutro.
4 estrellas: Positivo.
5 estrellas: Muy positivo.

In [16]:
text_class_pipe = pipeline("text-classification", model="nlptown/bert-base-multilingual-uncased-sentiment")
texto_clasificacion = "Este producto es excelente y de alta calidad."
resultado_clasificacion = text_class_pipe(texto_clasificacion)

print("Resultados para 'text-classification':")
print(resultado_clasificacion)
print()

Resultados para 'text-classification':
[{'label': '5 stars', 'score': 0.8085700869560242}]



# 3. Token Classification (Clasificación por token / NER)

Realiza reconocimiento de entidades nombradas (NER), etiquetando palabras como organizaciones, personas, lugares, etc.

In [17]:

token_class_pipe = pipeline("token-classification", model="mrm8488/bert-spanish-cased-finetuned-ner")
frase_token_class = "Apple está abriendo una nueva oficina en Bogotá."
resultados_token_class = token_class_pipe(frase_token_class)

print("Resultados para 'token-classification' (NER):")
for res in resultados_token_class:
    print(f"Entidad: {res['word']} | Tipo: {res['entity']} | Score: {res['score']:.4f}")
print()

Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Resultados para 'token-classification' (NER):
Entidad: Apple | Tipo: B-ORG | Score: 0.9971
Entidad: Bogotá | Tipo: B-LOC | Score: 0.9998



# 4. Text Generation (Generación de texto)

In [18]:

text_gen_pipe = pipeline("text-generation", model="datificate/gpt2-small-spanish")
frase_inicial_text_gen = "La inteligencia artificial es"
resultado_text_gen = text_gen_pipe(frase_inicial_text_gen, max_length=50, num_return_sequences=1)

print("Resultados para 'text-generation':")
for res in resultado_text_gen:
    print(res['generated_text'])
print()

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Resultados para 'text-generation':
La inteligencia artificial es una tecnología no invasiva que combina el control de campo, la detección de objetos y habilidades de sensores, y la capacidad de identificar y detectar objetos muy pequeños. Además, al control del campo se le hace un sistema de comunicación de



# 6. Summarization (Resumen de texto)

In [ ]:
summarization_pipe = pipeline("summarization", model="mrm8488/t5-base-finetuned-summarize-news")
texto_largo_summarization = """
La inteligencia artificial está transformando muchas industrias. Empresas como Google, Microsoft, y Amazon están invirtiendo en 
nuevas tecnologías para mejorar la eficiencia y la toma de decisiones automatizada. Estas innovaciones podrían cambiar la forma 
en que trabajamos y vivimos.
"""
resumen = summarization_pipe(texto_largo_summarization)

print("Resultados para 'summarization':")
print(resumen[0]['summary_text'])
print()

# 7. Translation (Traducción de texto)

In [19]:

translation_pipe = pipeline("translation_en_to_es", model="Helsinki-NLP/opus-mt-en-es")
texto_traduccion = "Artificial intelligence is revolutionizing many industries."
traduccion = translation_pipe(texto_traduccion)

print("Resultados para 'translation':")
print(traduccion[0]['translation_text'])
print()

ValueError: This tokenizer cannot be instantiated. Please make sure you have `sentencepiece` installed in order to use this tokenizer.

# 8. Sentence Similarity (Similitud de oraciones)

In [12]:

similarity_pipe = pipeline("feature-extraction", model="dccuchile/bert-base-spanish-wwm-uncased")
frase1 = "La inteligencia artificial transformará el futuro."
frase2 = "El futuro será transformado por la inteligencia artificial."
embedding1 = similarity_pipe(frase1)
embedding2 = similarity_pipe(frase2)
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Calcular la similitud entre las dos oraciones
embedding1_mean = np.mean(embedding1, axis=1)
embedding2_mean = np.mean(embedding2, axis=1)
similarity_score = cosine_similarity(embedding1_mean, embedding2_mean)

print("Resultados para 'sentence similarity':")
print(f"Similitud entre las oraciones: {similarity_score[0][0]:.4f}")
print()

Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Resultados para 'sentence similarity':
Similitud entre las oraciones: 0.8508



# Extracción de características con BETO - Convertir a una representación númerica
Este notebook demuestra cómo extraer características (embeddings) de un texto en español utilizando el modelo BETO, que es una versión de BERT entrenada en español.

In [22]:
from transformers import AutoTokenizer, AutoModel
import torch

# Cargar el modelo y el tokenizador BETO
model_name = "dccuchile/bert-base-spanish-wwm-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Texto en español para la extracción de características
text = "La inteligencia artificial está transformando el mundo."

# Tokenizar el texto
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

# Pasar los tokens por el modelo para obtener los embeddings
with torch.no_grad():
    outputs = model(**inputs)

# Extraer los embeddings del último hidden state (última capa del modelo)
embeddings = outputs.last_hidden_state

# Puedes elegir qué token o palabra específica utilizar (por ejemplo, el token CLS que representa el texto completo)
cls_embedding = embeddings[0, 0, :]  # Embedding del token [CLS]

# Imprimir el tamaño del embedding
print("Tamaño del embedding del token [CLS]:", cls_embedding.shape)

# Si quieres ver los embeddings para cada palabra en el texto
print("Embeddings para cada token en el texto:", embeddings.shape)


Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tamaño del embedding del token [CLS]: torch.Size([768])
Embeddings para cada token en el texto: torch.Size([1, 12, 768])


In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch

# Crear un pequeño DataFrame con frases en español
data = {'frases': [
    "La inteligencia artificial está transformando el mundo.",
    "Me gusta mucho programar en Python.",
    "El aprendizaje automático es fascinante.",
    "La banca está adoptando nuevas tecnologías.",
    "El análisis de datos es fundamental para la toma de decisiones."
]}

df = pd.DataFrame(data)


In [2]:
df

,frases
0,La inteligencia artificial está transformando ...
1,Me gusta mucho programar en Python.
2,El aprendizaje automático es fascinante.
3,La banca está adoptando nuevas tecnologías.
4,El análisis de datos es fundamental para la to...


In [3]:
# Cargar el modelo y el tokenizador BETO
model_name = "dccuchile/bert-base-spanish-wwm-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Función para extraer los embeddings del token [CLS] de una frase
def extract_features(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    cls_embedding = outputs.last_hidden_state[0, 0, :]  # Embedding del token [CLS]
    return cls_embedding.numpy()

# Aplicar la extracción de características a cada frase en el DataFrame
df['embeddings'] = df['frases'].apply(lambda x: extract_features(x))

Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
df

,frases,embeddings
0,La inteligencia artificial está transformando ...,"[0.08195162, -0.13856784, 0.23574236, -0.18076..."
1,Me gusta mucho programar en Python.,"[0.3350792, -0.7172838, 0.21734935, -0.6204882..."
2,El aprendizaje automático es fascinante.,"[0.03380447, -0.24542332, 0.28866348, -0.31269..."
3,La banca está adoptando nuevas tecnologías.,"[0.29102713, -0.15174729, 0.16909796, -0.36869..."
4,El análisis de datos es fundamental para la to...,"[-0.27688015, -0.010216851, 0.48100483, -0.049..."


In [7]:
import numpy as np
X = np.stack(df['embeddings'].values)

In [9]:
pd.DataFrame(X)

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.081952,-0.138568,0.235742,-0.180760,0.449923,0.317921,-0.120947,0.893363,-0.132951,-0.042052,...,-0.246324,-0.752582,-0.337168,-0.217546,-0.586880,-0.557141,-0.400848,-0.661943,0.035552,-0.278231
1,0.335079,-0.717284,0.217349,-0.620488,0.603344,-0.272233,-0.075453,-0.068326,-0.778578,0.054001,...,0.674065,-0.190477,-0.597092,0.697127,-1.249478,-0.521828,0.289186,-0.545779,-0.074797,-0.067943
2,0.033804,-0.245423,0.288663,-0.312693,0.751276,0.333301,0.152841,-0.141550,-0.491117,0.007622,...,0.544840,-0.553562,-0.009548,0.384719,-0.464000,-0.063875,-0.205182,-0.553683,-0.023206,-0.105826
3,0.291027,-0.151747,0.169098,-0.368696,0.423541,0.165202,0.507671,0.356805,-0.572448,-0.111768,...,-0.320171,-0.666623,-0.330645,0.249393,-0.162970,-0.191086,0.119498,0.246207,-0.119381,0.019641
4,-0.276880,-0.010217,0.481005,-0.049086,0.020754,-0.303803,0.035176,-0.257698,-0.334976,0.339052,...,0.573724,-0.565186,0.334923,0.809215,-0.671297,-0.092552,-0.130470,-0.280019,0.465952,0.657217


# Explicación suave 🫠

## 1. Instalación de las dependencias necesarias
Primero necesitamos instalar las librerías necesarias para cargar el modelo BETO.

In [ ]:
#!pip install transformers torch

## 2. Cargar el modelo BETO
A continuación, cargamos el modelo `BETO` y su tokenizador desde Hugging Face.

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

# Cargar el modelo y el tokenizador BETO
model_name = "dccuchile/bert-base-spanish-wwm-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 3. Introducir el texto en español para la extracción de características
Vamos a utilizar una frase en español como ejemplo para extraer sus características.

In [ ]:
# Texto en español para la extracción de características
text = "La inteligencia artificial está transformando el mundo."

# Tokenizar el texto
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

# Ver los tokens
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
print("Tokens generados:", tokens)
#Ver los ids
input_ids = tokenizer.convert_tokens_to_ids(tokens)
print(input_ids)

Tokens generados: ['[CLS]', 'La', 'inteligencia', 'artificial', 'está', 'transforma', '##n', '##do', 'el', 'mundo', '.', '[SEP]']
[4, 1198, 9145, 16061, 1266, 23684, 30935, 1047, 1040, 1863, 1009, 5]


## 4. Extraer los embeddings del texto tokenizado
El modelo procesará los tokens y generará embeddings para cada token en la frase.

In [ ]:
# Pasar los tokens por el modelo para obtener los embeddings
with torch.no_grad():
    outputs = model(**inputs)

# Extraer los embeddings del último hidden state (última capa del modelo)
embeddings = outputs.last_hidden_state

# Mostrar la forma de los embeddings
print("Forma de los embeddings:", embeddings.shape)

Forma de los embeddings: torch.Size([1, 12, 768])


## 5. Extraer el embedding
Los embeddings son representaciones numéricas que capturan el significado de las palabras en un espacio multidimensional. Estos embeddings se obtienen de la última capa del modelo y pueden ser utilizados para diversas tareas de NLP.

- **Embeddings por palabra**: Cada token tiene un embedding que lo representa en el contexto de la frase.
- **Embedding global**: El token `[CLS]` se utiliza como una representación global del significado de toda la frase.

La longitud máxima que BETO puede procesar es de 512 tokens, por lo que si tu texto excede ese límite, deberías dividirlo en partes.

In [ ]:
# Embedding del token [CLS] que representa la frase completa
cls_embedding = embeddings[0, 0, :]  # El primer [0] es la primera secuencia, el segundo [0] es el token [CLS]

# Mostrar el tamaño del embedding del token [CLS]
print("Tamaño del embedding del token [CLS]:", cls_embedding.shape)
print("Embedding del token [CLS]:", cls_embedding)

Tamaño del embedding del token [CLS]: torch.Size([768])
Embedding del token [CLS]: tensor([ 8.1952e-02, -1.3857e-01,  2.3574e-01, -1.8076e-01,  4.4992e-01,
         3.1792e-01, -1.2095e-01,  8.9336e-01, -1.3295e-01, -4.2052e-02,
        -1.3437e-01, -7.5474e-01, -1.9293e-01, -6.6547e-01,  3.9844e-01,
        -7.4332e-01, -1.2885e-01,  2.6310e-01, -5.3221e-01,  6.2915e-02,
        -3.1543e-01,  6.0861e-01, -7.2414e-02,  4.7553e-01, -8.4074e-01,
        -4.6770e-01,  4.7386e-01, -3.2404e-01, -3.1877e-01, -3.0246e-01,
         9.5490e-02, -2.2994e-01, -2.0117e-01,  5.6050e-01,  2.4203e-01,
        -6.7221e-01,  6.5838e-01,  3.7626e-02,  1.3419e-01, -2.6381e-02,
         4.8472e-01,  1.7211e-01,  1.4666e-01,  3.0293e-01, -1.6567e-01,
         7.1097e-01,  3.5347e-01,  1.6017e-01,  1.0213e+00, -7.5344e-05,
         2.8382e-01, -8.1311e-02, -7.1591e-01, -3.5693e-01,  3.4672e-01,
         4.4201e-02, -2.2630e-01, -6.6362e-01, -2.9959e-01, -4.3165e-02,
         5.8684e-02, -4.5498e-01, -3.3438

## 6. Embeddings por palabra
También podemos ver los embeddings para cada token de la frase.

In [ ]:
# Embeddings para cada token en la frase
for token, embedding in zip(tokens, embeddings[0]):
    print(f"Token: {token} -> Embedding: {embedding[:5]}... (truncado)")

Token: [CLS] -> Embedding: tensor([ 0.0820, -0.1386,  0.2357, -0.1808,  0.4499])... (truncado)
Token: La -> Embedding: tensor([ 0.0681,  0.6668, -0.7685, -0.2641,  0.3665])... (truncado)
Token: inteligencia -> Embedding: tensor([-0.1536, -0.4845, -0.4437,  0.1451, -0.2135])... (truncado)
Token: artificial -> Embedding: tensor([-0.5083, -0.5315,  0.1694, -0.4105,  0.2094])... (truncado)
Token: está -> Embedding: tensor([-0.1063,  0.2039,  0.0624, -0.3050,  0.4447])... (truncado)
Token: transforma -> Embedding: tensor([-0.4565,  0.3401, -0.7017,  0.3512, -0.0459])... (truncado)
Token: ##n -> Embedding: tensor([-0.5090,  0.5637, -0.9005,  0.3523,  0.3229])... (truncado)
Token: ##do -> Embedding: tensor([-0.4623,  0.4462, -1.0330,  0.2840,  0.3256])... (truncado)
Token: el -> Embedding: tensor([-0.8480,  0.8487, -0.4200,  0.7275, -0.1490])... (truncado)
Token: mundo -> Embedding: tensor([-0.5589,  0.2899, -0.1544,  0.3578, -0.2566])... (truncado)
Token: . -> Embedding: tensor([-0.3829,  0.